# QAOA Implementation: Number of rounds for solution

### Description

This notebook addresses the questions in Yigit's email concerning the number of QAOA rounds needed for three different linear systems. The matrix in each system is

\begin{equation}
    A := \frac{1}{6 k} \left[ 
        (3k + 3) II + (k - 1) IZ + (2k - 2) ZZ
        \right]
\end{equation}

where $k > 1$ is a real constant, $I$ and $Z$ are Pauli matrices, and $II := I \otimes I$ (similarly for the other terms). This matrix has the form

\begin{equation}
    A = \left[ \begin{matrix}
        1 & 0 & 0 & 0 \\
        0 & \frac{2k + 1}{3k} & 0 & 0 \\
        0 & 0 & \frac{k + 2}{3 k } & 0 \\
        0 & 0 & 0 & \frac{1}{k} \\
    \end{matrix} \right]
\end{equation}

The (unnormalized) vectors are

\begin{align}
    |b_1\rangle &:= |00\rangle + \frac{1}{k} |11\rangle \\
    |b_2\rangle &:= |00\rangle + \frac{2k + 1}{3k} |01\rangle + \frac{1}{k} |11\rangle \\
    |b_3\rangle &:= |00\rangle + \frac{2k + 1}{3k} |01\rangle + \frac{k + 2}{3k} \rangle |10\rangle + \frac{1}{k} |11\rangle 
\end{align}

_Note_: These vector labels (subscripts) start at one instead of starting at two (as in Yigit's notes).

**Question 1**: Can the solution $|x_j\rangle = A^{-1} |b_j\rangle$ can be prepared exactly by applying exactly $j + 1$ Hamiltonions in QAOA? The Hamiltonians are generated by

\begin{equation}
    H(s) := A(s) (I - |b\rangle \langle b| ) A(s)
\end{equation}

where $|b\rangle$ is one of $|b_j\rangle$ for $j = 1, 2, 3$ and

\begin{equation}
    A(s) := (1 - s) I + s A
\end{equation}

**Question 2**: Only the first system (with $|b_1\rangle$) can be prepared with two rounds of QAOA using $H(0)$ and $H(1)$. That is, $|x_2\rangle = A^{-1} |b_2\rangle$ and $|x_3\rangle = A^{-1} |b_3\rangle$ need more than two rounds of QAOA with $H(0)$ and $H(1)$ to be prepared. 

**Note**: We always start in the state $|b_j\rangle$ for system $j$. 

### Requirements

To run this notebook, the following packages are required:

* `numpy`
* `scipy`
* `matplotlib`

In [ ]:
"""Imports for the notebook."""
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize

### Helper functions and constants

Here we define functions for the tensor product and pure state fidelity.

In [ ]:
"""Helper functions for the notebook."""
def tensor(*args) -> np.ndarray:
    """Returns the tensor product over all matrices in args.
    
    Args:
        Each argument must be a np.ndarray.
    """
    if len(args) <= 0:
        raise ValueError("No arguments provided.")
    args = list(args)
    mat = args.pop(0)
    for term in args:
        mat = np.kron(mat, term)
    return mat

def fidelity(state1, state2):
    """Returns the fidelity for two (potentially unnormalized) pure states."""
    state1 /= np.linalg.norm(state1, ord=2)
    state2 /= np.linalg.norm(state2, ord=2)
    return abs(np.dot(state1, state2))**2

Here we define constants for the code. The value `k` defines the matrix, the value `FID` is the minimum fidelity for a solution, and `GRIDLEN` is the number of points along each axis in a grid search.

In [ ]:
"""Constants."""
# Fidelity of acceptable solutions
FID = 0.99

# Number of points along each axis in grid search. May be overriden in later cells.
GRIDLEN = 30

# Defining the matrix and vectors

First we define Pauli matrices and standard vectors needed for the matrix/vector definitions.

In [ ]:
"""Defining Pauli matrices and standard vectors."""
# Paulis
imat = np.identity(2)
zmat = np.array([[1, 0], [0, -1]])

# Standard vectors
zero = np.array([1, 0])
one = np.array([0, 1])

Now we define the matrix and vectors.

In [ ]:
"""Defining the matrix and vectors."""
# Matrix
def matrix(k):
    return ((3 * k + 3) * tensor(imat, imat) + 
            (k - 1) * tensor(imat, zmat) + 
            (2 * k - 2) * tensor(zmat, zmat)) / 6 / k

# Vectors
def vec1(k):
    vec = tensor(zero, zero) + tensor(one, one) / k
    vec /= np.linalg.norm(vec, ord=2)
    return vec

def vec2(k):
    vec = tensor(zero, zero) + (2 / 3 + 1 / 3 / k) * tensor(zero, one) + tensor(one, one) / k
    vec /= np.linalg.norm(vec, ord=2)
    return vec

def vec3(k):
    vec = tensor(zero, zero) + (2 / 3 + 1 / 3 / k) * tensor(zero, one) + \
           (1 / 3 + 2 / 3 / k) * tensor(one, zero) + tensor(one, one)
    vec /= np.linalg.norm(vec, ord=2)
    return vec

Now we perform some checks on the linear systems and display them.

In [ ]:
"""Small tests + print out the matrix and vectors."""
# Example value for k to test with
k = 2.0

# ==================================
# Test the eigenvalues of the matrix
# ==================================

# Analytic eigenvalues for the matrix
def analytic_evals(k):
    return [1.0, (2 * k + 1) / 3 / k, (k + 2) / 3 / k, 1 / k]

# Do the numerical diagonalization
evals, evecs = np.linalg.eigh(matrix(k))

# Check the eigenvalues are correct
assert np.allclose(list(sorted(evals)), list(sorted(analytic_evals(k))))

# ====================================
# Test that the vectors are normalized
# ====================================

for vec in (vec1(k), vec2(k), vec3(k)):
    assert np.isclose(np.linalg.norm(vec, ord=2), 1.0)

# ================================
# Print out the matrix and vectors
# ================================
print(f"For k = {k}, the matrix is\n")
print(matrix(k))
print("\nand the vectors are\n")
print(f"vec1 = {vec1(k)}\nvec2 = {vec2(k)}\nvec3 = {vec3(k)}")

# Defining the Hamiltonian

In [ ]:
"""Defining the Hamiltonian."""
def A(s, k):
    return (1 - s) * np.identity(len(matrix(k))) + s * matrix(k)

def H(s, b, k):
    return A(s, k) @ (np.identity(4) - np.outer(b, b)) @ A(s, k)

# Question 1: Solving the systems with given ansatz

In this section, we check that we can solve the systems with the specified ansatze.

### First system

For the $j = 1$ system we prepare

\begin{equation}
    |\hat{x}_1\rangle := e^{-i H(s_0) t_0} e^{-i H(s_1) t_1}  |b_1\rangle 
\end{equation}

where 

\begin{align}
    s_0 &= 0 \\
    s_1 &= 1 \\
\end{align}

and the parameters are in the range

\begin{align}
    t_0 &\in [0, 2\pi] \\
    t_1 &\in [0, 2 \pi k^2] \\
\end{align}

We check the fidelity of $|\hat{x}_1\rangle$ with the exact solution obtained classically $|x_1\rangle := A^{-1} |b_1\rangle$. 

In [ ]:
"""Solving the first system."""
# Number of points along each dimension in the grid search
GRIDLEN = 40

# Constants to determine the Hamiltonians
s0 = 0
s1 = 1

# Function to get the final state
def final_state(t0, t1, k):
    u1 = expm(-1j * H(s0, vec1(k), k) * t0)
    u2 = expm(-1j * H(s1, vec1(k), k) * t1)
    return u1 @ u2 @ vec1(k)

# Values of k to use
kvals = np.linspace(1.0, 10.0, 50)

# Empty list to store fidelities
fidelities = []

# Loop over each k value
for k in kvals:
    print("Status: k = {}, maximum k = {}".format(k, kvals[-1]))

    # Classical solution
    x1 = np.linalg.solve(matrix(k), vec1(k))

    # Initialize the fidelity
    fid = 0.0
    
    # Ranges for the parameters
    t0s = np.linspace(0, 2 * np.pi, GRIDLEN)
    t1s = np.linspace(0, 2 * np.pi * k**2, int(GRIDLEN * k**2))

    # Do the grid search
    for t0 in t0s:
        for t1 in t1s:
            fid = max(fid, fidelity(final_state(t0, t1, k), x1))
    
    # Store the maximum fidelity
    print("Grid search max fidelity obtained:", fid)

#     # Do the optimization
#     def cost(x):
#         val = fidelity(final_state(x[0], x[1], k), x1)
#         print(val)
#         return 1 - val
    
#     res = minimize(cost, x0=[0, 0], method="L-BFGS-B")
#     fid = 1.0 - res.fun

    # Store the maximum fidelity
    print("Maximum fidelity obtained:", fid)
    fidelities.append(fid)

In [ ]:
"""Plotting the fidelities obtained vs k value."""
plt.figure(figsize=(9, 5))
plt.rcParams.update({'font.size': 15, "font.weight": "bold"})
plt.plot(kvals, fidelities, linewidth=3.0, label="System: 1, QAOA Rounds: 2")
plt.grid()
plt.legend()
plt.xlabel("k");
plt.ylabel("Fidelity");
plt.ylim(0, 1)

### Second system

For the $j = 2$ system we prepare

\begin{equation}
    |\hat{x}_2\rangle := e^{-i H(s_0) t_0} e^{-i H(s_1) t_1} e^{-i H(s_2) t_2}  |b_2\rangle 
\end{equation}

where 

\begin{align}
    s_0 &= 0 \\
    s_1 &= \frac{k - \sqrt{k}}{k - 1} \\
    s_2 &= 1 \\
\end{align}

and the parameters are in the range

\begin{align}
    t_0 &\in [0, 2\pi] \\
    t_1 &\in [0, 2 \pi k] \\
    t_2 &\in [0, 2 \pi k^2] \\
\end{align}

We check the fidelity of $|\hat{x}_2\rangle$ with the exact solution obtained classically $|x_2\rangle := A^{-1} |b_2\rangle$. 

In [ ]:
"""Solving the second system with THREE rounds of QAOA."""
# Number of points along each dimension in the grid search
GRIDLEN = 20

# Constants to determine the Hamiltonians
s0 = 0
s1 = (k - np.sqrt(k)) / (k - 1)
s2 = 1

# Function to get the final state
def final_state3(t0, t1, t2, k):
    u1 = expm(-1j * H(s0, vec2(k), k) * t0)
    u2 = expm(-1j * H(s1, vec2(k), k) * t1)
    u3 = expm(-1j * H(s2, vec2(k), k) * t2)
    return u1 @ u2 @ u3 @ vec2(k)

# Values of k to use
kvals = np.linspace(1.01, 10.0, 50)

# Empty list to store fidelities
fidelities = []

# Loop over each k value
for k in kvals:
    print("Status: k = {}, maximum k = {}".format(k, kvals[-1]))

    # Classical solution
    x2 = np.linalg.solve(matrix(k), vec2(k))

    # Initialize the fidelity
    fid = 0.0
    
    # Ranges for the parameters
    t0s = np.linspace(0, 2 * np.pi, GRIDLEN)
    t1s = np.linspace(0, 2 * np.pi * k, int(GRIDLEN))
    t2s = np.linspace(0, 2 * np.pi * k**2, int(GRIDLEN))

    # Do the grid search
    for t0 in t0s:
        for t1 in t1s:
            for t2 in t2s:
                fid = max(fid, fidelity(final_state3(t0, t1, t2, k), x2))

    # Store the maximum fidelity
    print("Maximum fidelity obtained:", fid)
    fidelities.append(fid)

In [ ]:
"""Store the fidelities for system 2 with 3 rounds."""
system2fids_3rounds = fidelities

In [ ]:
"""Plotting the fidelities obtained vs k value."""
plt.figure(figsize=(9, 5))
plt.rcParams.update({'font.size': 15, "font.weight": "bold"})
plt.plot(kvals, system2fids_3rounds, linewidth=3.0, label="System: 2, QAOA Rounds: 3")
plt.grid()
plt.legend()
plt.xlabel("k");
plt.ylabel("Fidelity");

In [ ]:
"""Solving the second system with TWO rounds of QAOA."""
# Number of points along each dimension in the grid search
GRIDLEN = 40

# Constants to determine the Hamiltonians
s0 = 0
s1 = 1

# Function to get the final state
def final_state2(t0, t1, k):
    u1 = expm(-1j * H(s0, vec2(k), k) * t0)
    u2 = expm(-1j * H(s1, vec2(k), k) * t1)
    return u1 @ u2 @ vec2(k)

# Values of k to use
kvals = np.linspace(1.01, 10.0, 50)

# Empty list to store fidelities
fidelities = []

# Loop over each k value
for k in kvals:
    print("Status: k = {}, maximum k = {}".format(k, kvals[-1]))

    # Classical solution
    x2 = np.linalg.solve(matrix(k), vec2(k))

    # Initialize the fidelity
    fid = 0.0
    
    # Ranges for the parameters
    t0s = np.linspace(0, 2 * np.pi, GRIDLEN)
    t1s = np.linspace(0, 2 * np.pi * k**2, int(GRIDLEN))

    # Do the grid search
    for t0 in t0s:
        for t1 in t1s:
            fid = max(fid, fidelity(final_state3(t0, t1, t2, k), x2))

    # Store the maximum fidelity
    print("Maximum fidelity obtained:", fid)
    fidelities.append(fid)

In [ ]:
"""Store the fidelities for system 2 with 2 rounds."""
system2fids_2rounds = fidelities

In [ ]:
"""Plotting the fidelities obtained vs k value."""
plt.figure(figsize=(9, 5))
plt.rcParams.update({'font.size': 15, "font.weight": "bold"})
plt.plot(kvals, system2fids_2rounds, linewidth=3.0, label="System: 2, QAOA Rounds: 2")
plt.grid()
plt.legend()
plt.xlabel("k");
plt.ylabel("Fidelity");

In [ ]:
"""Plot the fidelities for both 2 rounds and 3 rounds on the same plot."""
plt.figure(figsize=(9, 5))
plt.rcParams.update({'font.size': 15, "font.weight": "bold"})
plt.plot(kvals, system2fids_2rounds, linewidth=3.0, label="System: 2, QAOA Rounds: 2")
plt.plot(kvals, system2fids_3rounds, linewidth=3.0, label="System: 2, QAOA Rounds: 3")
plt.grid()
plt.legend()
plt.xlabel("k");
plt.ylabel("Fidelity");

### Third system

For the $j = 3$ system we prepare

\begin{equation}
    |\hat{x}_3\rangle := e^{-i H(s_0) t_0} e^{-i H(s_1) t_1} e^{-i H(s_2) t_2} e^{-i H(s_3) t_3}  |b_2\rangle 
\end{equation}

where 

\begin{align}
    s_0 &= 0 \\
    s_1 &= \frac{k - k^{2/3}}{k - 1} \\
    s_2 &= \frac{k - k^{1/3}}{k - 1} \\
    s_3 &= 1 \\
\end{align}

and the parameters are in the range

\begin{align}
    t_0 &\in [0, 2\pi] \\
    t_1 &\in [0, 2 \pi k^{2 / 3}] \\
    t_2 &\in [0, 2 \pi k^{4 / 3}] \\
    t_3 &\in [0, 2 \pi k^2] \\
\end{align}

We check the fidelity of $|\hat{x}_3\rangle$ with the exact solution obtained classically $|x_3\rangle := A^{-1} |b_3\rangle$.

In [ ]:
"""Solving the third system with TWO rounds of QAOA."""
# Number of points along each dimension in the grid search
GRIDLEN = 30

# Constants to determine the Hamiltonians
s0 = 0
s1 = 1

# Function to get the final state
def final_state2(t0, t1, k):
    u1 = expm(-1j * H(s0, vec3(k), k) * t0)
    u2 = expm(-1j * H(s1, vec3(k), k) * t1)
    return u1 @ u2 @ vec3(k)

# Values of k to use
kvals = np.linspace(1.01, 10.0, 50)

# Empty list to store fidelities
fidelities = []

# Loop over each k value
for k in kvals:
    print("Status: k = {}, maximum k = {}".format(k, kvals[-1]))

    # Classical solution
    x3 = np.linalg.solve(matrix(k), vec3(k))

    # Initialize the fidelity
    fid = 0.0
    
    # Ranges for the parameters
    t0s = np.linspace(0, 2 * np.pi, GRIDLEN)
    t1s = np.linspace(0, 2 * np.pi * k**2, int(GRIDLEN))

    # Do the grid search
    for t0 in t0s:
        for t1 in t1s:
            fid = max(fid, fidelity(final_state3(t0, t1, t2, k), x3))

    # Store the maximum fidelity
    print("Maximum fidelity obtained:", fid)
    fidelities.append(fid)

In [ ]:
"""Store the fidelities for system 3 with 2 rounds."""
system3fids_2rounds = fidelities

In [ ]:
"""Plot system three with two rounds of QAOA."""
plt.figure(figsize=(9, 5))
plt.rcParams.update({'font.size': 15, "font.weight": "bold"})
plt.plot(kvals, system3fids_2rounds, linewidth=3.0, label="System: 3, QAOA Rounds: 2")
plt.grid()
plt.legend()
plt.xlabel("k");
plt.ylabel("Fidelity");

In [ ]:
"""Solving the third system with THREE rounds of QAOA."""
# Number of points along each dimension in the grid search
GRIDLEN = 20

# Constants to determine the Hamiltonians
# Constants to determine the Hamiltonians
s0 = 0
s1 = (k - np.sqrt(k)) / (k - 1)
s2 = 1

# Function to get the final state
def final_state3(t0, t1, t2, k):
    u1 = expm(-1j * H(s0, vec3(k), k) * t0)
    u2 = expm(-1j * H(s1, vec3(k), k) * t1)
    u3 = expm(-1j * H(s2, vec3(k), k) * t2)
    return u1 @ u2 @ u3 @ vec3(k)

# Values of k to use
kvals = np.linspace(1.01, 10.0, 50)

# Empty list to store fidelities
fidelities = []

# Loop over each k value
for k in kvals:
    print("Status: k = {}, maximum k = {}".format(k, kvals[-1]))

    # Classical solution
    x3 = np.linalg.solve(matrix(k), vec3(k))

    # Initialize the fidelity
    fid = 0.0
    
    # Ranges for the parameters
    t0s = np.linspace(0, 2 * np.pi, GRIDLEN)
    t1s = np.linspace(0, 2 * np.pi * k, int(GRIDLEN))
    t2s = np.linspace(0, 2 * np.pi * k**2, int(GRIDLEN))

    # Do the grid search
    for t0 in t0s:
        for t1 in t1s:
            for t2 in t2s:
                fid = max(fid, fidelity(final_state3(t0, t1, t2, k), x3))

    # Store the maximum fidelity
    print("Maximum fidelity obtained:", fid)
    fidelities.append(fid)

In [ ]:
"""Store the fidelities for system 3 with 3 rounds."""
system3fids_3rounds = fidelities

In [ ]:
"""Plot system three with three rounds (and two rounds) of QAOA."""
plt.figure(figsize=(9, 5))
plt.rcParams.update({'font.size': 15, "font.weight": "bold"})
plt.plot(kvals, system3fids_2rounds, linewidth=3.0, label="System: 3, QAOA Rounds: 2")
plt.plot(kvals, system3fids_3rounds, linewidth=3.0, label="System: 3, QAOA Rounds: 3")
plt.grid()
plt.legend()
plt.xlabel("k");
plt.ylabel("Fidelity");

In [ ]:
"""Solving the third system with FOUR rounds of QAOA."""
# Number of points along each dimension in the grid search
GRIDLEN = 20

# Constants to determine the Hamiltonians
s0 = 0
s1 = (k - k**(2/3)) / (k - 1)
s2 = (k - k**(1/3)) / (k - 1)
s3 = 1

# Function to get the final state
def final_state4(t0, t1, t2, t3):
    u1 = expm(-1j * H(s0, vec3(k)) * t0)
    u2 = expm(-1j * H(s1, vec3(k)) * t1)
    u3 = expm(-1j * H(s2, vec3(k)) * t2)
    u4 = expm(-1j * H(s3, vec3(k)) * t3)
    return u1 @ u2 @ u3 @ u4 @ vec3(k)

# Values of k to use
kvals = np.linspace(1.01, 10.0, 50)

# Empty list to store fidelities
fidelities = []

# Loop over each k value
for k in kvals:
    print("Status: k = {}, maximum k = {}".format(k, kvals[-1]))

    # Classical solution
    x3 = np.linalg.solve(matrix(k), vec3(k))

    # Initialize the fidelity
    fid = 0.0
    
    # Ranges for the parameters
    t0s = np.linspace(0, 2 * np.pi, GRIDLEN)
    t1s = np.linspace(0, 2 * np.pi * k**(2/3), GRIDLEN)
    t2s = np.linspace(0, 2 * np.pi * k**(4/3), GRIDLEN)
    t3s = np.linspace(0, 2 * np.pi * k**2, GRIDLEN)

    # Do the grid search
    for t0 in t0s:
        for t1 in t1s:
            for t2 in t2s:
                for t3 in t3s:
                    fid = max(fid, fidelity(final_state3(t0, t1, t2, k), x3))

    # Store the maximum fidelity
    print("Maximum fidelity obtained:", fid)
    fidelities.append(fid)

In [ ]:
"""Store the fidelities for system 3 with 4 rounds."""
system3fids_4rounds = fidelities